# Collect coinmarketcap ids and links (slug) using API

https://jibrelnetwork.atlassian.net/browse/JWWEB-256
https://jibrelnetwork.atlassian.net/browse/JWWEB-255

In [ ]:
ASSETS_FILE = '../../jwallet-assets/mainnet/assets.json'

Request mapping from api. There are ~1800 items available so, sooner or later, pagination will be required.

In [ ]:
import requests
map_url = "https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/map"
key = 'ef850729-cae3-426a-99bd-3969f72d98dd'
response = requests.get(map_url, params={"limit": 5000}, headers={
    'X-CMC_PRO_API_KEY': key
})

In [ ]:
data = response.json()

In [ ]:
import json
with open(ASSETS_FILE) as fp:
    assets = json.load(fp)

In [ ]:
assets_symbols = set([a['symbol'] for a in assets])

In [ ]:
matches = [d for d in data['data'] if d['symbol'] in assets_symbols]

In [ ]:
for item in assets:
    for matched in matches:
        if item['symbol'] == matched['symbol']:
            if 'priceFeed' not in item:
                item['priceFeed'] = {}
            # JWWEB-255: Collect coinmarketcap IDs
            item['priceFeed']['currencyID'] = matched['id']
            item['priceFeed']['currencyIDType'] = "coinmarketcap"
            # JWWEB-256
            currency_url = "https://coinmarketcap.com/currencies/%s" % matched['slug']
            url_type = 'coinmarketcap'
            if 'assetPage' not in item:
                item['assetPage'] = {'urls': []}
            for url in item['assetPage'].get('urls', []):
                if url['type'] == url_type:
                    url['url'] = currency_url
                    break
            else:
                item['assetPage']['urls'].append({'type': url_type, 'url': currency_url})

In [ ]:
with open(ASSETS_FILE, 'w') as fp:
    json.dump(assets, fp, indent=2, ensure_ascii=False)